One of your homework assignments was to write a method that finds twos. It takes two strings and returns any elements that appears in both lists containing the digit 2.

Here is a pretty terrible implementation of `find_twos`:

In [ ]:
def find_twos(first_list, second_list):
    return []

This is not a great implementation of `find_twos`, clearly :). **You do not have to make find_twos in this notebook work correctly.** But we can use this poor implementation of the function to understand and evaluate what tests look like when they pass and fail.

When we write the tests before the code, that is called **Test Driven Development (TDD)**.

How would we make sure that `find_twos` is working?

### This is one way:

In [ ]:
def test_find_twos():
    assert(find_twos([], []) == [])
    assert(find_twos([2], [12]) == [])
    assert(find_twos([12], [12]) == [12])
    assert(find_twos([12, 2], [12]) == [12])
    assert(find_twos([12, 2, 3], [12, 2]) == [12, 2])
    assert(find_twos([12, 3], [12, 2]) == [12])
    assert(find_twos([1, 3, 4], [1, 3, 4]) == [])
    
test_find_twos()

**What are some of the problems with that way?**

### OK, how about this way? Is this better?

In [ ]:
def test(function, examples):
    passed = 0
    run = 0

    for example in examples:
        run += 1
        expected = example[-1]
        actual = function(*example[:-1])

        if expected == actual:
            passed += 1
        else:
            print(f"Whoops. For example {example}, the function returned {actual}.")

    print(f"\n{passed} out of {run} examples worked as expected.")

find_twos_examples = [
    ("", "", []),
    ("1", "1, 3", []),
    ("2", "", []),
    ("2", "1, 3", []),
    ("2", "2", [2]),
    ("2", "12", []),
    ("12", "2, 12", [12]),
    ("1, 3, 5, 12, 7, 200", "2, 6, 9, 200, 5", [200]),
    ("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2", [2, 22]),
    ("1,2, 20,22, 44, 99", "3,5, 22, 100, 44, 2", [2, 22]),
    ("1,2, 20,22, 22,44, 20, 99", "3,5, 22, 100, 44, 2", [2, 22]),
    ("1, 2, 20, 22", "3, 2, 20, 22", [2, 20, 22]),
]

test(function=find_twos, examples=find_twos_examples)

**What do you like about this way? What don't you like about this way?**

### It's time to talk about the first critical component of a test framework: matchers.

What if we could get clearer output from our tests? Suppose we could do something like this for `find_twos`:

```
>>> assert_that(find_twos("", "")).equals([])
>>> True

>>> assert_that(find_twos("", "")).equals(["wrong answer"])
>>> FailedAssertion: Expected ['wrong answer'] but got []
```

Wouldn't that be nicer?

Let's see if we can create an API like that. Here's some code to get you started: 

In [ ]:
class FailedAssertion(Exception):
    pass

class Assertion:
    def __init__(self, expression):
        self.expression = expression

    def equals(self, expected_result):
        '''
            This method should return True if the assertion works
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        pass

def assert_that(expression):
    return Assertion(expression)

In [ ]:
assert_that(2 + 2).equals(4) # Should pass

In [ ]:
assert_that(1 + 1).equals(9) # Should fail

The `assert_that` and `equals` methods that we have called above are called **matchers**. Test frameworks often have a wide variety of matchers that programmers like you or I can use to write expressions describing all the things we want to check in our tests. We will write more matchers a little later, but `.equals()` is a great one to start with.

We can use our `.equals()` matcher to write tests for `find_twos` like this:

In [ ]:
    def test_equals_passing():
        assert_that(2 + 2).equals(4)
        
    def test_equals_failing():
        assert_that(1 + 1).equals(9)

In [ ]:
test_equals_pasing()
test_equals_failing()

### Challenge:

Implement the `equals` matcher. When it is working, the first test above should _pass_, and the second test should raise a `FailedAssertion` with a message like "expected 9 but got 2."

### Okay, so. 

Now let's get a little more sophisticated. We've got one matcher, and it's a pretty useful matcher, so we can write a lot of tests with it. But what might be helfpul is to have some more matchers to make our tests more clear and expressive. For example, what if we had an `is_empty` matcher? Then we could do:

In [ ]:
    def test_is_empty_passing():
        assert_that([]).is_empty()

    def test_is_empty_failing():
        assert_that([2, 3, 4]).is_empty()

In [ ]:
test_is_empty_passing()
test_is_empty_failing()

### Challenge 

Implement the `is_empty` matcher.

Below see a block of example code to help you get started on that:

In [ ]:
# You can technically have a class subclass itself. 
# I have done so here to avoid copy-pasting the implementation of Assertion over.
# We are now adding methods to the assertion class.
# In Swift, by the way, there's a keyword for doing exactly this: extension.
class Assertion(Assertion): 
    def is_empty(self):
        '''
            This method should return True if a collection has no items
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        pass

In [ ]:
test_is_empty_passing()
test_is_empty_failing()

After you implement `is_empty()`, the first test should pass and the second test should raise a `FailedAssertion` with a message like "Expected [2, 3, 4] to be empty."

Good, good. Our test framework is coming along swimmingly. 

One thing to know about tests is, when we're testing collections, it is often helpful to make sure the collection is the right _size_ before we go into a bunch of complicated assertions about what's _in_ it. With a nice matcher, we could do that like:

In [ ]:
    def test_has_size_passing():
        assert_that([]).has_size(0)
        assert_that([3]).has_size(1)

    def test_has_size_failing():
        assert_that([4, 5, 6]).has_size(10)

In [ ]:
test_has_size_passing()
test_has_size_failing()

### Challenge: 

Implement the `has_size` assertion.

In [ ]:
class Assertion(Assertion): 
    def has_size(self, size):
        '''
            This method should return True if a collection has the right number of items
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        pass

In [ ]:
test_has_size_passing()
test_has_size_failing()

When you run _these_ tests, the first one should pass and the second one should raise a `FailedAssertion` with a message like "Expected size 10, but got size 3." 

Let's do just one more today. One of the annoying things about our implementation right now is that `find_twos` has to return results in a specific _order_ for the assertion to pass. For example, if we check:

```
assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])
```

And `find_twos` returns `[22, 2]`, it will fail. 

What if we don't care about order? What if we just want to make sure all the right items are in the collection, regardless of order? Then we could do:

In [ ]:
    def test_has_items_passing():
        assert_that([2, 22]).has_items(22)
        assert_that([2, 22]).has_items(22, 2)

    def test_has_items_failing():
        assert_that([2, 22]).has_items(44)

In [ ]:
test_has_items_passing()
test_has_items_failing()

### Challenge:

Implement the `has_items` assertion.

NOTE THAT the implementation above doesn't require the items to be passed in as a collection. Instead, the programmer writing the test can just pass in as many items to this method as they want.

In [ ]:
class Assertion(Assertion): 
    def has_items(self, *args):
        '''
            This method should return True if a collection has the right items
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        pass

In [ ]:
test_has_items_passing()
test_has_items_failing()

Once you've implemented `has_items`, the first test above should pass and the second should raise a `FailedAssertion` with a message like "Expected input to have item 44."

## Great Job!

So far, we have implemented one critical component of a test library: the **matchers**.

Next up, we'll implement the other critical component of a test library: the **runner**.